In [39]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sys
from PIL import Image
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Convolution2D, Dropout, Input, GlobalAveragePooling2D, MaxPooling2D, Flatten, Dense# Sequential model for building a linear stack of layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
#!unzip drive/My\ Drive/images.zip

# Code


In [32]:
sys.path.append('/content/drive/MyDrive/face2face/functions')

In [33]:
#from create_dataframe import create_df

In [35]:
train_meta = create_df('train')
valid_meta = create_df('valid')
test_meta = create_df('test')

NameError: ignored

In [36]:
# Specify the base path where your image data is stored
base_path = '/content/real_vs_fake/real-vs-fake/'


# Create an ImageDataGenerator for image preprocessing and augmentation
image_gen = ImageDataGenerator(rescale=1./255.,
                               rotation_range=20,
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True,
                               fill_mode='nearest')
# The 'rescale' parameter normalizes pixel values to the range [0, 1] by dividing each pixel value by 255

In [37]:
Resnet50_train = image_gen.flow_from_directory(
    base_path + 'train/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)
Resnet50_valid = image_gen.flow_from_directory(
    base_path + 'valid/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)
Resnet50_test = image_gen.flow_from_directory(
    base_path + 'test/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [40]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94765736/94765736 [==============================] - 4s 0us/step


In [41]:
for layer in base_model.layers:
  layer.trainable=False

In [42]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 64)                131136    
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 23718913 (90.48 MB)
Trainable params: 131201 (512.50 KB)
Non-trainable params: 23587712 (89.98 MB)
_________

In [43]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [46]:
model.fit(Resnet50_train, epochs = 11, validation_data=Resnet50_valid)

Epoch 1/11
1563/1563 [==============================] - 1416s 902ms/step - loss: 0.6927 - accuracy: 0.5183 - val_loss: 0.6893 - val_accuracy: 0.5135
Epoch 2/11
1563/1563 [==============================] - 1387s 887ms/step - loss: 0.6881 - accuracy: 0.5410 - val_loss: 0.6854 - val_accuracy: 0.5585
Epoch 3/11
1563/1563 [==============================] - 1368s 875ms/step - loss: 0.6859 - accuracy: 0.5511 - val_loss: 0.6818 - val_accuracy: 0.5696
Epoch 4/11
1563/1563 [==============================] - 1385s 886ms/step - loss: 0.6855 - accuracy: 0.5530 - val_loss: 0.6833 - val_accuracy: 0.5581
Epoch 5/11
1563/1563 [==============================] - 1372s 878ms/step - loss: 0.6849 - accuracy: 0.5554 - val_loss: 0.6856 - val_accuracy: 0.5246
Epoch 6/11
1563/1563 [==============================] - 1376s 880ms/step - loss: 0.6835 - accuracy: 0.5582 - val_loss: 0.6820 - val_accuracy: 0.5591
Epoch 7/11
1563/1563 [==============================] - 1370s 876ms/step - loss: 0.6826 - accuracy: 0.5618

In [47]:
model.save('/content/drive/MyDrive/face2face/models/ResNet50.keras')

In [48]:
Res_model = load_model('/content/drive/MyDrive/face2face/models/ResNet50.keras')

In [70]:
Res_model.evaluate(Resnet50_test)

313/313 [==============================] - 220s 697ms/step - loss: 0.6839 - accuracy: 0.5523


[0.6839320659637451, 0.5523499846458435]

In [79]:
predictions = Res_model.predict(Resnet50_test)

313/313 [==============================] - 216s 686ms/step


In [52]:
y_pred = (predictions > 0.5).astype(int)

In [53]:
y_true = Resnet50_test.classes

In [86]:
print(model.history.history)

{}


In [57]:
plt.plot(model.history.history['loss'], label='train')
plt.plot(model.history.history['val_loss'], label='test')
plt.legend()
plt.title('Loss')
plt.show()

KeyError: ignored

In [61]:
plt.plot(model.history.history['accuracy'], label='train')
plt.plot(model.history.history['val_accuracy'], label='test')
plt.legend()
plt.title('Loss')
plt.show()

KeyError: ignored

In [87]:
from sklearn.metrics import confusion_matrix, classification_report

In [88]:
confusion_matrix(y_true, y_pred)

array([[8362, 1638],
       [8344, 1656]])

In [89]:
Resnet50_metrics = classification_report(y_true, y_pred)

In [90]:
print(Resnet50_metrics)

              precision    recall  f1-score   support

           0       0.50      0.84      0.63     10000
           1       0.50      0.17      0.25     10000

    accuracy                           0.50     20000
   macro avg       0.50      0.50      0.44     20000
weighted avg       0.50      0.50      0.44     20000

